In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import estimator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import load_wine
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
wine_data = load_wine()

In [3]:
print(wine_data['DESCR'])

Wine Data Database

Notes
-----
Data Set Characteristics:
    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- 1) Alcohol
 		- 2) Malic acid
 		- 3) Ash
		- 4) Alcalinity of ash  
 		- 5) Magnesium
		- 6) Total phenols
 		- 7) Flavanoids
 		- 8) Nonflavanoid phenols
 		- 9) Proanthocyanins
		- 10)Color intensity
 		- 11)Hue
 		- 12)OD280/OD315 of diluted wines
 		- 13)Proline
        	- class:
                - class_0
                - class_1
                - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash: 

In [4]:
feat_data = wine_data['data']
labels = pd.get_dummies(wine_data['target']).as_matrix()

In [5]:
X_train, X_test, y_train, y_test = train_test_split(feat_data, labels, test_size=0.3, random_state=101)

In [6]:
scaler = MinMaxScaler()

In [7]:
scaler_x_train = scaler.fit_transform(X_train)
scaler_x_test = scaler.transform(X_test)

In [8]:
dataset = tf.data.Dataset.from_tensor_slices((scaler_x_train, y_train))

In [9]:
batch_size = 20
num_of_features = 13
num_of_outputs = 3

In [10]:
dataset = dataset.repeat(None)
dataset = dataset.shuffle(buffer_size=100)
dataset = dataset.batch(batch_size)

In [11]:
iterator = dataset.make_initializable_iterator()
x_batch, y_batch = iterator.get_next()

In [12]:
x = tf.placeholder(tf.float32, shape=[None, num_of_features])
y_true = tf.placeholder(tf.int32, shape=[None, num_of_outputs])

In [13]:
W = tf.Variable(tf.truncated_normal(shape=[num_of_features, num_of_outputs]))
b = tf.Variable(tf.random_normal(shape=[num_of_outputs]))

In [14]:
y = tf.matmul(x, W) + b

In [15]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true, logits=y)
)

In [16]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train = optimizer.minimize(cross_entropy)

In [17]:
init = tf.global_variables_initializer()

In [18]:
steps = 10_000

with tf.Session() as sess:
    sess.run(init)
    sess.run(iterator.initializer)
    
    for i in range(steps):
        x_val, y_val = sess.run([x_batch, y_batch])

        sess.run(train, feed_dict={
            x: x_val,
            y_true: y_val
        })

    weight, bias = sess.run([W, b])
 
    predictions = [np.argmax(np.matmul(data, weight) + bias)
                   for data in scaler_x_test]

In [19]:
original_vals = [np.argmax(data) for data in y_test]

In [20]:
print(classification_report(original_vals, predictions))

             precision    recall  f1-score   support

          0       0.95      1.00      0.97        19
          1       1.00      0.95      0.98        22
          2       1.00      1.00      1.00        13

avg / total       0.98      0.98      0.98        54

